In [4]:
### --- imports ---
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [5]:
#   Class for Meander line coordinates
#   Can return r_hat vectors for each dipole orientation

class CoordinateSet:
    def __init__(self, coords, lengths):
        self.coords = np.array(coords, dtype=float) # shape (N,3) -----> Maybe add the length so we can calc based on length of each side (some are smaller)
        self.lengths = np.array(lengths, dtype=float)

    def as_array(self):
        #Return the (N,3) numpy array of (x,y,z).
        return self.coords
    
    def lengths_array(self):
        # Return the lengths
        return self.lengths
    
    def r_vectors(self):
        #Return full 3D vectors (x,y,z) as (N,3).
        return self.coords
    
    def r_norms(self):
        #Return distances from origin (N,).
        return np.linalg.norm(self.r_vectors(), axis=1)
    
    def r_hat(self):
        #Return unit vectors pointing from origin to each (x,y,z)
        r_vecs = self.r_vectors()
        return r_vecs / self.r_norms()[:,None]
    
coords = [[1,0,0], [0,1,0], [0,0,1]]    # x-axis, y-axis, z-axis
lengths = [0.01, 0.01, 0.05]            # 1cm, 1cm, 5cm

meander = CoordinateSet(coords, lengths)

print(meander.as_array())
print(meander.lengths_array())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0.01 0.01 0.05]


Assuming a uniform microstrip with per-unit parameters collapsed into: <br>
    $Z_0 \\$
    $\gamma = \alpha + j\beta$

The current along a lossy matched transmission line is a single forward traveling wave: <br>
$I(\ell) = I_0 e^{-j\gamma \ell}$, <br>
where $I_0$ is the current at the input terminals and $\ell$ is the length of the transmission (meander) line

In [6]:
#   Current distribution along transmission line
#   Derived from transmission line theory
#   Matched load assumed → forward-traveling wave only (no reflections)

# Lossy transmission line
def currentDistLossy(I_0, l, alpha, beta):
    return I_0 * np.exp(-(alpha + 1j*beta)*l)

# Lossless transmission line
def currentDistLossless(I_0, l, alpha, beta):
    return I_0 * np.exp(-1j*beta*l)

The E-field from an infinitesimal dipole antenna:  

$$
E_{r} = \eta \frac{I_0 l \cos(\theta)}{2\pi r^2} \left[ 1 + \frac{1}{jkr} \right] e^{-jkr}
$$

$$
E_{\theta} = j \eta \frac{k I_0 l \sin(\theta)}{4 \pi r} \left[ 1 + \frac{1}{jkr} - \frac{1}{(kr)^2} \right] e^{-jkr}
$$

$$
E_{\phi} = 0
$$

We know:

$$
\underline{\hat{r}} = \underline{\hat{s}}
$$

$$
\underline{\hat{\phi}} = \frac{\underline{\hat{h}} \times \underline{\hat{s}}}{\lVert \underline{\hat{h}} \times \underline{\hat{s}} \rVert}
$$

$$
\underline{\hat{\theta}} = \frac{(\underline{\hat{s}} \cdot \underline{\hat{h}}) \underline{\hat{s}} - \underline{\hat{h}}}{\lVert (\underline{\hat{s}} \cdot \underline{\hat{h}}) \underline{\hat{s}} - \underline{\hat{h}} \rVert}
$$

Also:

$$
R' = \lVert \underline{s} \rVert
$$

$$
\lVert \underline{\hat{h}} \times \underline{\hat{s}} \rVert = \sin \theta
$$

$$
\underline{\hat{s}} \cdot \underline{\hat{h}} = \cos \theta
$$

And:

$$
\underline{V}_{r\theta\phi} =
\begin{bmatrix}
V_r \\
V_\theta \\
V_\phi
\end{bmatrix}
=
\begin{bmatrix}
\sin\theta \cos\phi & \sin\theta \sin\phi & \cos\theta \\
\cos\theta \cos\phi & \cos\theta \sin\phi & -\sin\theta \\
-\sin\phi & \cos\phi & 0
\end{bmatrix}
\begin{bmatrix}
V_x \\
V_y \\
V_z
\end{bmatrix}
$$

where $\hat{h}$ = dipole axis unit vector, $\hat{s}$ = normalized direction vector from the Hertz dipole to the observer point.


In [ ]:
# --- Get field from single dipole with length l at a single observation point R---
def singleDipole(I0, l, r, h_hat, s_hat):
    c0 = 299_792_458    # Speed of light
    eta = 120*np.pi     # Free-space permittivity
    freq = 1030e6       # Frequency from paper
    Lambda = c0/freq    # Free-space wavelength 
    k = 2*np.pi/Lambda    
    pass
"""
Must have:
    - eta:   Permittivity
    - k:     2pi/lambda
    - I0:    Current at point l on transmission line (use currentDist functions)
    - l:     Length of infinitesimal dipole
    - r:     Radius from dipole (origin) to observation point R
    - h_hat: Unit vector along dipole axis
    - s_hat: Unit vector pointing toward observation point R    
"""
# Test